# About this notebook

This notebook assumes you have ran the local Iris classification notebook ("1 Local End to End") and you have not deleted the LOCAL_ROOT folder. In this notebook, we will deploy a trained model and submit prediction requests.

# Setting things up

In [1]:
import mltoolbox.classification.dnn as sd

No handlers could be found for logger "oauth2client.contrib.multistore_file"


In [2]:
import os
import pandas as pd
import tensorflow as tf
from tensorflow.python.lib.io import file_io

This notebook will write files during prediction. Please give a root folder you wish to use.

In [3]:
LOCAL_ROOT = './iris_notebook_workspace' # This should be the same as what was used in the local notebook
CLOUD_ROOT = 'gs://' + datalab_project_id() + '-iris-classification-datalab' # Feel free to change this line.

# No need to edit anything else in this cell.
LOCAL_TRAINING_DIR = os.path.join(LOCAL_ROOT, 'training')
CLOUD_TRAINING_DIR = os.path.join(CLOUD_ROOT, 'training')

if not file_io.file_exists(LOCAL_ROOT):
  raise ValueError('LOCAL_ROOT not found. Did you run the local notebook?')
  
!gsutil mb {CLOUD_ROOT}

Creating gs://cloud-ml-dev-iris-classification-datalab/...
ServiceException: 409 Bucket cloud-ml-dev-iris-classification-datalab already exists.


First, let us put the csv files on GCS and the output of training.

In [4]:
!gsutil -m cp -r {LOCAL_TRAINING_DIR} {CLOUD_TRAINING_DIR}

Copying file://./iris_notebook_workspace/training/features_file.json [Content-Type=application/json]...
Copying file://./iris_notebook_workspace/training/model/saved_model.pb [Content-Type=application/octet-stream]...
Copying file://./iris_notebook_workspace/training/model/variables/variables.index [Content-Type=application/octet-stream]...
Copying file://./iris_notebook_workspace/training/model/variables/variables.data-00000-of-00001 [Content-Type=application/octet-stream]...
Copying file://./iris_notebook_workspace/training/model/assets.extra/schema.json [Content-Type=application/json]...
Copying file://./iris_notebook_workspace/training/evaluation_model/variables/variables.index [Content-Type=application/octet-stream]...
Copying file://./iris_notebook_workspace/training/evaluation_model/variables/variables.data-00000-of-00001 [Content-Type=application/octet-stream]...
Copying file://./iris_notebook_workspace/training/model/assets.extra/vocab_flower.csv [Content-Type=text/csv]...
Cop

In [5]:
!gsutil ls {CLOUD_TRAINING_DIR}

gs://cloud-ml-dev-iris-classification-datalab/training/features_file.json
gs://cloud-ml-dev-iris-classification-datalab/training/evaluation_model/
gs://cloud-ml-dev-iris-classification-datalab/training/model/
gs://cloud-ml-dev-iris-classification-datalab/training/train/
gs://cloud-ml-dev-iris-classification-datalab/training/training/


# Online Prediction using ML Engine

We first have to deploy the model, and then we can make requests on it.

In [6]:
!gcloud beta ml models create iris_model --regions=us-central1
!gcloud beta ml versions create v1 --model iris_model --origin {CLOUD_ROOT}/training/model

Creating version (this might take a few minutes)......done.


In [7]:
pandas_df = sd.predict(
  cloud=True,
  model_name='iris_model',
  model_version='v1',
  data=['101,6.3,3.3,6,2.5', #virginica
        '100,5.7,2.8,4.1,1.3', #versicolor
        '13,4.8,3,1.4,0.1', #setosa
  ]
)
pandas_df

,key,top_1_label,top_1_score,top_2_label,top_2_score,top_3_label,top_3_score
0,101,Iris-virginica,0.999999,Iris-versicolor,1.29123e-06,Iris-setosa,1.26783e-15
1,100,Iris-versicolor,0.999751,Iris-virginica,0.000224599,Iris-setosa,2.48638e-05
2,13,Iris-setosa,0.999748,Iris-versicolor,0.000252439,Iris-virginica,9.99411e-10


In [8]:
pandas_df = sd.predict(
  cloud=True,
  model_name='iris_model',
  model_version='v1',
  data=pd.DataFrame(
    [[101,6.3,3.3,6,2.5],
     [100,5.7,2.8,4.1,1.3],
     [13,4.8,3,1.4,0.1]])
)
pandas_df

,key,top_1_label,top_1_score,top_2_label,top_2_score,top_3_label,top_3_score
0,101,Iris-virginica,0.999999,Iris-versicolor,1.29123e-06,Iris-setosa,1.26783e-15
1,100,Iris-versicolor,0.999751,Iris-virginica,0.000224599,Iris-setosa,2.48638e-05
2,13,Iris-setosa,0.999748,Iris-versicolor,0.000252439,Iris-virginica,9.99411e-10


Cleaning things up
=====

If you want to delete the files you made on GCS, uncomment and run the next cell.

In [9]:
#!gsutil rm -fr {CLOUD_ROOT}

The next cell deletes the deployed model. Don't run it if you want to keep it around.

In [ ]:
!gcloud beta ml versions delete v1 --model iris_model 
!gcloud beta ml models delete iris_model 